In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter 
from sklearn.preprocessing import scale
from modAL.density import information_density
np.random.seed(42)
import math

In [8]:
'''
ft:挑选后的特征
t:特征对应金标准
name:样本名称
c: 特征对应簇
# '''
df1 = pd.read_csv('../FeatureBank/ISIC2018_NPID.csv',header = None)
# print(df1.shape)
ft = df1.values #特征样本/256维
# ft = scale(ft) #进行归一化

df2 = pd.read_csv('../2048/train.txt',sep = ' ',header = None)#一般不修
name = df2.iloc[:,0].values #图片位置名称
# t = df2.iloc[:,1].values #金标准0/1
# print('Original dataset:', Counter(t))

df3 = pd.read_csv('../kmeans/ISIC18_NPID_kmeans++.csv',header = 0)
c = df3.iloc[:,1].values
print('Clustering:', Counter(c))

Clustering: Counter({3: 244, 5: 233, 2: 220, 4: 218, 7: 216, 9: 211, 8: 207, 0: 202, 6: 165, 1: 159})


In [13]:
n_clusters = 10
ratio=0.25
total_num = len(c)
sel_num= math.ceil(ratio*total_num/n_clusters)
sel_num

52

In [14]:
from __future__ import print_function
import torch
from torch.autograd import Variable

def farthest_point_sample(xyz, npoint): 

    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """

#     xyz = xyz.transpose(2,1)
    device = xyz.device
    B, N, C = xyz.shape
    
    centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)     # 采样点矩阵（B, npoint）
    distance = torch.ones(B, N).to(device) * 1e10                       # 采样点到所有点距离（B, N）

    batch_indices = torch.arange(B, dtype=torch.long).to(device)        # batch_size 数组

    #farthest = torch.randint(0, N, (B,), dtype=torch.long).to(device)  # 初始时随机选择一点
    
    barycenter = torch.sum((xyz), 1)                                    #计算重心坐标 及 距离重心最远的点
    barycenter = barycenter/xyz.shape[1]
    barycenter = barycenter.view(B, 1, C)

    dist = torch.sum((xyz - barycenter) ** 2, -1)
    farthest = torch.max(dist,1)[1]                                     #将距离重心最远的点作为第一个点

    for i in range(npoint):
#         print("-------------------------------------------------------")
#         print("The %d farthest pts %s " % (i, farthest))
        centroids[:, i] = farthest                                      # 更新第i个最远点
        centroid = xyz[batch_indices, farthest, :].view(B, 1, C)        # 取出这个最远点的xyz坐标
        dist = torch.sum((xyz - centroid) ** 2, -1)                     # 计算点集中的所有点到这个最远点的欧式距离
#         print("dist    : ", dist)
        mask = dist < distance
#         print("mask %i : %s" % (i,mask))
        distance[mask] = dist[mask]                                     # 更新distance，记录样本中每个点距离所有已出现的采样点的最小距离
#         print("distance: ", distance)

        farthest = torch.max(distance, -1)[1]                           # 返回最远点索引
 
    return centroids

# if __name__ == '__main__':

#     sim_data = Variable(torch.rand(2,10000,128))
#     print(sim_data)

#     centroids = farthest_point_sample(sim_data, 100)
    
#     print("Sampled pts: ", centroids)

In [15]:
'''
Maxcover
'''
import math
from sklearn.ensemble import IsolationForest

index=[] #index[i]表示第i个聚类对应的index列表
ft_part=[] #ft_part[i]表示第i个聚类对应的特征向量列表
new_index=[] #新列表

for i in range(n_clusters):
    index.append(np.where(c == i)[0])
    ft_part.append(torch.Tensor([ft[index[i]]]))

for i in range(n_clusters):
#     #注意：比例
# #     clf = IsolationForest(contamination=ratio,n_jobs=20).fit(ft_part[i])
    
# #     novel_pred = clf.predict(ft_part[i])
# #     li = np.where(novel_pred==-1)
    
# #     novel_scores = clf.score_samples(ft_part[i])
# #     li = np.argsort(novel_scores)[:sel_num]



    centroids = farthest_point_sample(ft_part[i], sel_num).numpy().flatten()
#     print("Sampled pts: ", centroids)
    new_index.extend(index[i][centroids])
    
ft_sel = ft[new_index]
# t_sel = t[new_index]
c_sel = c[new_index]
name_sel = name[new_index]

from collections import Counter 
print('Resampled dataset:', Counter(c_sel))
print(len(c_sel))

Resampled dataset: Counter({0: 52, 1: 52, 2: 52, 3: 52, 4: 52, 5: 52, 6: 52, 7: 52, 8: 52, 9: 52})
520


In [16]:
'''
注意保存csv的后缀
'''
print(__doc__)
dataframe1 = pd.DataFrame({'image_name':name_sel})
dataframe1.to_csv("../txt/NPID/ISIC_FPS25.txt",index=False,sep=' ',header = None)


注意保存csv的后缀

